# 16장

In [ ]:
import tensorflow as tf, os
import warnings
warnings.filterwarnings('ignore')

tf.get_logger().setLevel('ERROR')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가 방지 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
os.putenv('TF_GPU_ALLOCATOR', 'cuda_malloc_async')

In [2]:
import numpy as np, tensorflow as tf
from tensorflow import keras

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[ 4  5  6  7]
 [10 11 12 13]
 [ 6  7  8  9]]
===== 
Y_batch
[[ 5  6  7  8]
 [11 12 13 14]
 [ 7  8  9 10]]
____________________ Batch 1 
X_batch
[[ 8  9 10 11]
 [ 2  3  4  5]
 [ 0  1  2  3]]
===== 
Y_batch
[[ 9 10 11 12]
 [ 3  4  5  6]
 [ 1  2  3  4]]


In [2]:
for i in tf.data.Dataset.from_tensor_slices(tf.range(15)):
    print(i)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)


In [3]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
print(tokenizer.texts_to_sequences(['First']))
print(tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(['First'])))
max_id = len(tokenizer.word_index)  # 고유한 문자 갯수
dataset_size = tokenizer.document_count # 전체 문자 갯수
print(max_id, dataset_size)
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) -1   # 1~39를 0~38로 변환하기 위해 -1
print(encoded.shape)

[[20, 6, 9, 8, 3]]
['f i r s t']
39 1115394
(1115394,)


In [5]:
# train_size = dataset_size * 90 // 100
train_size = dataset_size * 90 // 50000
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
print(train_size, dataset_size)

2007 1115394


In [6]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [7]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [8]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
                    #  dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
                    #  dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
60/60 [==============================] - 51s 808ms/step - loss: 3.1075
Epoch 2/10
60/60 [==============================] - 53s 872ms/step - loss: 2.5708
Epoch 3/10
60/60 [==============================] - 72s 1s/step - loss: 2.1977
Epoch 4/10
60/60 [==============================] - 89s 1s/step - loss: 1.9609
Epoch 5/10
60/60 [==============================] - 69s 1s/step - loss: 1.7232
Epoch 6/10
60/60 [==============================] - 77s 1s/step - loss: 1.4866
Epoch 7/10
60/60 [==============================] - 102s 2s/step - loss: 1.2604
Epoch 8/10
60/60 [==============================] - 76s 1s/step - loss: 1.0565
Epoch 9/10
60/60 [==============================] - 93s 2s/step - loss: 0.8922
Epoch 10/10
60/60 [==============================] - 102s 2s/step - loss: 0.7565


In [9]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
#                      #dropout=0.2, recurrent_dropout=0.2),
#                      dropout=0.2),
#     keras.layers.GRU(128, return_sequences=True,
#                      #dropout=0.2, recurrent_dropout=0.2),
#                      dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id,
#                                                     activation="softmax"))
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(dataset, batch_size=1024, epochs=10)

In [10]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [11]:
X_new = preprocess(["How are yo"])
y_pred = np.argmax(model(X_new), axis=-1)
print(X_new)
print()
print(y_pred)
print(tokenizer.sequences_to_texts(y_pred + 1)[0][-1])

tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [12]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1

    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    
    return text

In [13]:
print(complete_text('t', temperature=0.2))
print('='*30)
print(complete_text('w', temperature=1))
print('='*30)
print(complete_text('w', temperature=2))

the object of our misery, is a gain to the commonal
w't, we know't.

first citizen:
he hath famousl, is
wzif3ricuull. be doney proud; sofkmry, goodeco3tt!



In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [65]:
# batch_size = 32
# encoded_parts = np.array_split(encoded[:train_size], batch_size)
# datasets = []
# for encoded_part in encoded_parts:
#     dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
#     dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(window_length))
#     datasets.append(dataset)
# dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
# dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
# dataset = dataset.map(
#     lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
# dataset = dataset.prefetch(1)

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2, 
                     batch_input_shape=[1, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
20/20 [==============================] - 9s 373ms/step - loss: 3.4010
Epoch 2/50
20/20 [==============================] - 8s 415ms/step - loss: 3.0855
Epoch 3/50
20/20 [==============================] - 9s 433ms/step - loss: 3.0326
Epoch 4/50
20/20 [==============================] - 20s 1s/step - loss: 2.9773
Epoch 5/50
20/20 [==============================] - 25s 1s/step - loss: 2.8936
Epoch 6/50
20/20 [==============================] - 27s 1s/step - loss: 2.7868
Epoch 7/50
20/20 [==============================] - 38s 2s/step - loss: 2.6691
Epoch 8/50
20/20 [==============================] - 18s 880ms/step - loss: 2.5556
Epoch 9/50
20/20 [==============================] - 14s 689ms/step - loss: 2.4478
Epoch 10/50
20/20 [==============================] - 10s 477ms/step - loss: 2.4099
Epoch 11/50
20/20 [==============================] - 10s 499ms/step - loss: 2.3616
Epoch 12/50
20/20 [==============================] - 10s 490ms/step - loss: 2.3143
Epoch 13/50
20/20 [=========

In [68]:
print(complete_text('t', temperature=0.2))
print('='*30)
print(complete_text('w', temperature=1))
print('='*30)
print(complete_text('w', temperature=2))

tizen: what he hamane the cannond to the cations.


wo reseak, net ento to patrly.

second citizen:
ins
wwt thay i
fkak seir3 tsecwlvldg
ne: pao!
cudp:ygak


In [3]:
import tensorflow_datasets as tfds

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(('<pad>', '<sos>', '<unk>')):
    id_to_word[id_] = token

print(" ".join(id_to_word[id_] for id_ in X_train[0][:10]))


datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

(25000,) (25000,) (25000,) (25000,)
<sos> this film was just brilliant casting location scenery story


In [4]:
from collections import Counter

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b'<br\\s*/?>', b' ')
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b' ')
    X_batch = tf.strings.split(X_batch)

    return X_batch.to_tensor(default_value=b'<pad>'), y_batch

vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

print(vocabulary.most_common()[:3])

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]


In [5]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [6]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [10]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)    

Epoch 1/5
782/782 [==============================] - 46s 54ms/step - loss: 0.5383 - accuracy: 0.7187
Epoch 2/5
782/782 [==============================] - 41s 53ms/step - loss: 0.3407 - accuracy: 0.8566
Epoch 3/5
782/782 [==============================] - 42s 54ms/step - loss: 0.1800 - accuracy: 0.9363
Epoch 4/5
782/782 [==============================] - 38s 49ms/step - loss: 0.1372 - accuracy: 0.9507
Epoch 5/5
782/782 [==============================] - 35s 45ms/step - loss: 0.1162 - accuracy: 0.9579


In [11]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation='sigmoid')(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 46s 53ms/step - loss: 0.5296 - accuracy: 0.7269
Epoch 2/5
782/782 [==============================] - 42s 54ms/step - loss: 0.3477 - accuracy: 0.8549
Epoch 3/5
782/782 [==============================] - 39s 50ms/step - loss: 0.1994 - accuracy: 0.9300
Epoch 4/5
782/782 [==============================] - 39s 50ms/step - loss: 0.1369 - accuracy: 0.9517
Epoch 5/5
782/782 [==============================] - 39s 50ms/step - loss: 0.1129 - accuracy: 0.9599


In [39]:
import tensorflow_hub as hub

model = keras.models.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/nnlm-en-dim50/2',
                   dtype=tf.string, input_shape=[], output_shape=50),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples
batch_size = 32
train_set = datasets['train'].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5457 - accuracy: 0.7280
Epoch 2/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5140 - accuracy: 0.7490
Epoch 3/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5090 - accuracy: 0.7526
Epoch 4/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5056 - accuracy: 0.7543
Epoch 5/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5027 - accuracy: 0.7552


In [40]:
model = keras.Sequential([
    # trainable=True로 할 경우 코랩에서 메모리 부족 에러가 발생합니다.
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=False, dtype=tf.string, input_shape=[]),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=10)

Epoch 1/10
782/782 [==============================] - 19s 22ms/step - loss: 0.3764 - accuracy: 0.8409
Epoch 2/10
782/782 [==============================] - 18s 22ms/step - loss: 0.3253 - accuracy: 0.8597
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3204 - accuracy: 0.8616
Epoch 4/10
782/782 [==============================] - 19s 24ms/step - loss: 0.3163 - accuracy: 0.8644
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3124 - accuracy: 0.8665
Epoch 6/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3084 - accuracy: 0.8684
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3045 - accuracy: 0.8710
Epoch 8/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3004 - accuracy: 0.8728
Epoch 9/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2962 - accuracy: 0.8752
Epoch 10/10
782/782 [==============================] - 18s 23ms/step - loss: 0.292

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(decoder_embeddings, initial_state=encoder_state, 
                                                             sequence_length=sequence_lengths)
y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths], outputs=[y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

X = np.random.randint(100, size=10*1000).reshape(1000, 10)
y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], y, epochs=2)

In [43]:
# beam_width = 10
# decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
#     cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)
# decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(encoder_state, multiplier=beam_width)
# outputs, _, _ = decoder(embedding_decoder, start_tokens=start_tokens, end_token=end_token, 
#                         initial_state=decoder_initial_state)

In [7]:
from pathlib import Path

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # 쌍을 2개의 리스트로 분리합니다.

for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

I think it was a misunderstanding. => Creo que fue un malentendido.
He makes it a rule never to speak ill of others. => Él tiene por regla nunca hablar mal de otros.
The new tunnel is twice as long as the old one. => El nuevo túnel es el doble de largo que el viejo.


In [9]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

print(text_vec_layer_en.get_vocabulary()[:10])
print(text_vec_layer_es.get_vocabulary()[:10])

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']
['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']


In [10]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

encoder_inputs = keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = keras.layers.Input(shape=[], dtype=tf.string)

embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

decoder = keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

output_layer = keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

In [11]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input 
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

print(translate("I like soccer"))
print('='*30)
print(translate("I like soccer and also going to the beach"))

1/1 [==============================] - 0s 30ms/step
cosas llegar ese sólo cayó así padres hacer seguir hacer bastante bastante suficiente banco bastante ayer música música pienso pena pena perro conmigo estar más somos comprado café cabeza por por profesor profesor di completamente abrió mira profesor mucho seis único acaba minutos entiendo vista minutos vista minutos boca invierno
1/1 [==============================] - 0s 32ms/step
alta rompió pensar rompió saber tren beber pero de quien largo guerra venga hacia siendo ayudar ayudar mes odio odio


In [ ]:
encoder = keras.layers.Bidirectional(
    keras.layers.LSTM(256, return_state=True))
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # 단기 상태 (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # 장기 상태 (1 & 3)
# 추가 코드 - 모델을 완성하고 학습시킵니다.
decoder = keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))
print(translate("I like soccer"))
print('='*30)
print(translate("I like soccer and also going to the beach"))

In [12]:
# 추가 코드 - 빔 검색의 기본 구현

def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # 인코더 입력
    X_dec = np.array(["startofseq"])  # 디코더 입력
    y_proba = model.predict((X, X_dec))[0, 0]  # 첫 번째 토큰의 확률
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # 촤상의 (log_proba, translation) 리스트 
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]
    
    # 추가 코드 - verbose 모드에서 상위 첫 단어를 표시합니다.
    if verbose:
        print("상위 첫 단어:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # 번역이 완료되었으므로 번역을 이어가지 않습니다.
            X = np.array([sentence_en])  # 인코더 입력
            X_dec = np.array(["startofseq " + translation])  # 디코더 입력
            y_proba = model.predict((X, X_dec))[0, idx]  # 마지막 토큰의 확률
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # 추가 코드 - verbose 모드의 경우 지금까지의 최상의 번역을 출력합니다.
        if verbose:
            print("지금까지 최상의 번역:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

# 추가 코드 - 모델이 어떻게 오류를 발생시키는지 보여줍니다.
sentence_en = "I love cats and dogs"
print(translate(sentence_en))
print('='*30)
print(beam_search(sentence_en, beam_width=3, verbose=True))

1/1 [==============================] - 0s 35ms/step
comer corriendo abrir amor voz llevó listo ayuda inglés tienes colegio encontró ir ir cuesta quien algún las mismo contigo mismo podemos tres tres sos futuro futuro grande muchas pena haber manos precio iré iré policía mía comer ningún oportunidad montaña tenés cuántas zapatos zapatos zapatos comprado amiga amiga zapatos
1/1 [==============================] - 0s 34ms/step
상위 첫 단어: [(-6.8989854, 'comer'), (-6.899154, 'zapatos'), (-6.8993587, 'les')]
1/1 [==============================] - 0s 28ms/step
지금까지 최상의 번역: [(-13.793785, 'zapatos corriendo'), (-13.795052, 'zapatos comer'), (-13.796391, 'zapatos zapatos')]
1/1 [==============================] - 0s 33ms/step
지금까지 최상의 번역: [(-20.687904, 'zapatos zapatos corriendo'), (-20.690207, 'zapatos zapatos comer'), (-20.690353, 'zapatos comer corriendo')]
1/1 [==============================] - 0s 35ms/step
지금까지 최상의 번역: [(-27.582058, 'zapatos zapatos corriendo pensaba'), (-27.583464, 'zapatos za

In [ ]:
encoder = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True, return_state=True))

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),
                 tf.concat(encoder_state[1::2], axis=-1)]
decoder = keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

attention_layer = keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = keras.layers.Dense(vocab_size, activation='softmax')
y_proba = output_layer(attention_outputs)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

In [52]:
print(translate("I like soccer and also going to the beach"))
print('='*30)
print(beam_search("I like soccer and also going to the beach", beam_width=3, verbose=True))

1/1 [==============================] - 0s 28ms/step
[UNK] y [UNK] a [UNK] a [UNK]
1/1 [==============================] - 0s 27ms/step
상위 첫 단어: [(-1.9274566, '[UNK]'), (-2.2856333, 'me'), (-3.6605964, 'quiero')]
1/1 [==============================] - 0s 28ms/step
지금까지 최상의 번역: [(-3.9298213, '[UNK] y'), (-3.9538238, '[UNK] [UNK]'), (-4.079806, 'me [UNK]')]
1/1 [==============================] - 0s 27ms/step
지금까지 최상의 번역: [(-5.5314198, '[UNK] y [UNK]'), (-6.087785, 'me [UNK] y'), (-6.1089306, 'me [UNK] [UNK]')]
1/1 [==============================] - 0s 44ms/step
지금까지 최상의 번역: [(-7.4136157, '[UNK] y [UNK] a'), (-7.6505456, 'me [UNK] y [UNK]'), (-8.026182, 'me [UNK] [UNK] a')]
1/1 [==============================] - 0s 33ms/step
지금까지 최상의 번역: [(-9.002893, '[UNK] y [UNK] a [UNK]'), (-9.391769, 'me [UNK] y [UNK] a'), (-9.575634, 'me [UNK] [UNK] a [UNK]')]
1/1 [==============================] - 0s 31ms/step
지금까지 최상의 번역: [(-10.744028, '[UNK] y [UNK] a [UNK] a'), (-10.990112, 'me [UNK] y [UNK] a [UNK

In [13]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims%2 == 1:
            max_dims += 1   # max_dims는 짝수여야 함
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims//2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    
    def call(self, inputs):
        shape = tf.shape(inputs)
        
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [14]:
embed_size = 512
max_steps = 500
vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

Z = encoder_in
for N in range(6):
    X = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    # Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, Z], use_causal_mask=True)    # tensorflow WARNING에 따라 변경
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(keras.layers.Dense(vocab_size, activation='softmax'))(Z)

In [21]:
max_length = 50  # 전체 훈련 세트에 있는 최대 길이
embed_size = 128

encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

pos_embed_layer = keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

In [22]:
N = 2
num_heads = 8
dropout_rate = 0.1
n_units = 128
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))
    
    skip = Z
    Z = keras.layers.Dense(n_units, activation='relu')(Z)
    Z = keras.layers.Dense(embed_size)(Z)
    Z = keras.layers.Dropout(dropout_rate)(Z)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))

In [ ]:
batch_max_len_dec = 50  # tf.shape(decoder_embeddings)[1]
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, None]
causal_mask = tf.linalg.band_part(  # 행렬의 대각선 부분을 추출하거나 수정하는 함수
    tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

encoder_outputs = Z
Z = decoder_in
for _ in range(N):
    skip = Z
    attn_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask&decoder_pad_mask)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))
    skip = Z

    attn_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))
    skip = Z

    Z = keras.layers.Dense(n_units, activation='relu')(Z)
    Z = keras.layers.Dense(embed_size)(Z)
    Z = keras.layers.LayerNormalization()(keras.layers.Add()([Z, skip]))

y_proba = keras.layers.Dense(vocab_size, activation='softmax')(Z)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

In [26]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")  # 다른 많은 작업을 사용할 수 있습니다.
result = classifier("The actors were very convincing.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [27]:
classifier(["I am from India.", "I am from Iraq."])

[{'label': 'POSITIVE', 'score': 0.9896161556243896},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]

In [ ]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli("She loves me. [SEP] She loves me not.")

In [29]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [30]:
token_ids = tokenizer(["I like soccer. [SEP] We all love soccer!",
                       "Joe lived for a very long time. [SEP] Joe is old."],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]])>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [31]:
token_ids = tokenizer([("I like soccer.", "We all love soccer!"),
                       ("Joe lived for a very long time.", "Joe is old.")],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]])>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [32]:
outputs = model(token_ids)
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-2.1123827 ,  1.1786797 ,  1.4101014 ],
       [-0.01478171,  1.0962448 , -0.9919954 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [33]:
Y_probas = tf.keras.activations.softmax(outputs.logits)
Y_probas

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.01619699, 0.43523583, 0.5485672 ],
       [0.22656046, 0.6881716 , 0.08526791]], dtype=float32)>

In [34]:
Y_pred = tf.argmax(Y_probas, axis=1)
Y_pred  # 0 = contradiction, 1 = entailment, 2 = neutral

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1], dtype=int64)>

In [35]:
sentences = [("Sky is blue", "Sky is red"), ("I love her", "She loves me")]
X_train = tokenizer(sentences, padding=True, return_tensors="tf").data
y_train = tf.constant([0, 2])  # contradiction, neutral
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2)

Epoch 1/2
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 31s 31s/step - loss: 1.5941 - accuracy: 0.5000
Epoch 2/2
1/1 [==============================] - 0s 247ms/step - loss: 0.3027 - accuracy: 1.0000


In [27]:
import numpy as np, keras, tensorflow as tf

MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

MONTHS_DICT = {f'{i:>02}': v for i, v in enumerate(MONTHS, 1)}
print(MONTHS_DICT)

start_date = np.datetime64('1000-01-01')
end_date = np.datetime64('9999-12-31')
y = np.arange(start_date, end_date + np.timedelta64(1, 'D'), dtype='datetime64[D]')
np.random.shuffle(y)
y = list(map(str, y))
X = list(map(lambda x: MONTHS_DICT[x[5:7]] + ' ' + x[-2:] + ', ' + x[:4], y))
print(X[0], len(X), y[0], len(y))

{'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May', '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November', '12': 'December'}
April 04, 8436 3287182 8436-04-04 3287182


In [28]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
# MONTHS를 join 후 set으로 중복 문자 제거한 후 숫자와 쉼표, 공백을 포함하여 입력값 원소를 정렬하여 문자열로 생성
print(INPUT_CHARS)
OUTPUT_CHARS = "0123456789-"    # 출력값 원소를 문자열로 생성

 ,0123456789ADFJMNOSabceghilmnoprstuvy


In [29]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    """date_str에서 INPUT_CHARS 인덱스로 ID를 생성하는 함수"""
    return [chars.index(c) for c in date_str]
print(date_str_to_ids(X[0], INPUT_CHARS))
print(date_str_to_ids(y[0], OUTPUT_CHARS))

[12, 31, 32, 26, 27, 0, 2, 6, 1, 0, 10, 6, 5, 8]
[8, 4, 3, 6, 10, 0, 4, 10, 0, 4]


In [43]:
from functools import partial

def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # 0을 패딩 토큰 ID로 사용

X = list(map(prepare_date_strs, X))

prepare_date_strs = partial(prepare_date_strs, chars=OUTPUT_CHARS)
y = list(map(prepare_date_strs, y))